#### Names of people in the group

Please write the names of the people in your group in the next cell.

Anne Torgersen

Aaryan Neupane

In [ ]:
# Deleting tables left from previous runs in case they still exist after deleting an inactive cluster
dbutils.fs.rm("/user", recurse=True)

Out[6]: True

In [ ]:
# We need to install 'ipython_unittest' to run unittests in a Jupyter notebook
%pip install -q ipython_unittest

In [ ]:
# Loading PySpark modules that we need
import unittest
from collections import Counter
from pyspark.sql import DataFrame
from pyspark.sql.types import *
from pyspark.sql import SparkSession

#### Subtask 1: defining the schema for the data
Typically, the first thing to do before loading the data into a Spark cluster is to define the schema for the data. Look at the schema for 'badges' and try to define the schema for other tables similarly.

In [ ]:
# Defining a schema for 'badges' table
badges_schema = StructType([StructField('UserId', IntegerType(), False),
                            StructField('Name', StringType(), False),
                            StructField('Date', TimestampType(), False),
                            StructField('Class', IntegerType(), False)])

# Defining a schema for 'posts' table
posts_schema = StructType([StructField('Id', IntegerType(), False),
                            StructField('ParentId', IntegerType(), False),    
                           StructField('PostTypeId', IntegerType(), False),
                            StructField('CreationDate', TimestampType(), False),
                            StructField('Score', IntegerType(), False), 
                           StructField('ViewCount', IntegerType(), False),
                           StructField('Body', IntegerType(), False),
                           StructField('OwnerUserId', IntegerType(), False),
                          StructField('LastActivityDate', TimestampType(), False),
                          StructField('Title', StringType(), False),
                          StructField('Tags', StringType(), False),
                          StructField('AnswerCount', IntegerType(), False),
                          StructField('CommentCount', IntegerType(), False),
                          StructField('FavoriteCount', IntegerType(), False),
                          StructField('CloseDate', TimestampType(), False)])

# Defining a schema for 'users' table
users_schema = StructType([StructField('Id', IntegerType(), False),
                            StructField('Reputation', StringType(), False),
                            StructField('CreationDate', TimestampType(), False),
                            StructField('DisplayName', StringType(), False), 
                           StructField('LastAccessDate', TimestampType(), False),
                          StructField('AboutMe', StringType(), False),
                          StructField('Views', IntegerType(), False),
                          StructField('UpVotes', IntegerType(), False),
                          StructField('DownVotes', IntegerType(), False)])

# Defining a schema for 'comments' table
comments_schema = StructType([StructField('PostId', IntegerType(), False),
                            StructField('Score', IntegerType(), False),
                    StructField('Text', StringType(), False),
                            StructField('CreationDate', TimestampType(), False), 
                           StructField('UserId', IntegerType(), False)])

#### Subtask 2: implementing two helper functions
Next, we need to implement two helper functions:
1. 'load_csv' that as input argument receives path for a CSV file and a schema and loads the CSV pointed by the path into a Spark DataFrame and returns the DataFrame;
2. 'save_df' receives a Spark DataFrame and saves it as a Parquet file on DBFS.

Note that the column separator in CSV files is TAB character ('\t') and the first row includes the name of the columns. 

BTW, DBFS is the name of the distributed filesystem used by Databricks Community Edition to store and access data.

In [ ]:
def load_csv(source_file: "path for the CSV file to load", schema: "schema for the CSV file being loaded as a DataFrame") -> DataFrame:
    df = spark.read.csv(source_file, schema=schema, sep="\t", header=True)
    return df

def save_df(df: "DataFrame to be saved", table_name: "name under which the DataFrame will be saved") -> None:
    df.write.format("parquet").mode('overwrite').saveAsTable(table_name)

In [ ]:
# Loading 'ipython_unittest' so we can use '%%unittest_main' magic command
%load_ext ipython_unittest

#### Subtask 3: validating the implementation by running the tests

Run the cell below and make sure that all the tests run successfully. Moreover, at the end there should be four Parquet files named 'badges', 'comments', 'posts', and 'users' in '/user/hive/warehouse'.

Note that we assumed that the data for the project has already been stored on DBFS on the '/FileStore/tables/' path. (I mean as 'badges_csv.gz', 'comments_csv.gz', 'posts_csv.gz', and 'users_csv.gz'.)

In [ ]:
%%unittest_main
class TestTask1(unittest.TestCase):
   
    # test 1
    def test_load_badges(self):
        result = load_csv(source_file="/FileStore/tables/badges_csv.gz", schema=badges_schema)
        self.assertIsNotNone(result, "Badges dataframe did not load successfully")
        self.assertIsInstance(result, DataFrame, "Result type is not of spark.sql.DataFrame")
        self.assertEqual(result.count(), 105640, "Number of records is not correct")

        coulmn_names = Counter(map(str.lower, ['UserId', 'Name', 'Date', 'Class']))
        self.assertCountEqual(coulmn_names, Counter(map(str.lower, result.columns)),
                              "Missing column(s) or column name mismatch")
    
    # test 2
    def test_load_posts(self):
        result = load_csv(source_file="/FileStore/tables/posts_csv.gz", schema=posts_schema)
        self.assertIsNotNone(result, "Posts dataframe did not load successfully")
        self.assertIsInstance(result, DataFrame, "Result type is not of spark.sql.DataFrame")
        self.assertEqual(result.count(), 61432, "Number of records is not correct")

        coulmn_names = Counter(map(str.lower,
                                   ['Id', 'ParentId', 'PostTypeId', 'CreationDate', 'Score', 'ViewCount', 'Body', 'OwnerUserId',
                                    'LastActivityDate', 'Title', 'Tags', 'AnswerCount', 'CommentCount', 'FavoriteCount',
                                    'CloseDate']))
        self.assertCountEqual(coulmn_names, Counter(map(str.lower, result.columns)),
                              "Missing column(s) or column name mismatch")
    
    # test 3
    def test_load_comments(self):
        result = load_csv(source_file="/FileStore/tables/comments_csv.gz", schema=comments_schema)
        self.assertIsNotNone(result, "Comments dataframe did not load successfully")
        self.assertIsInstance(result, DataFrame, "Result type is not of spark.sql.DataFrame")
        self.assertEqual(result.count(), 58735, "Number of records is not correct")

        coulmn_names = Counter(map(str.lower, ['PostId', 'Score', 'Text', 'CreationDate', 'UserId']))
        self.assertCountEqual(coulmn_names, Counter(map(str.lower, result.columns)),
                              "Missing column(s) or column name mismatch")
    
    # test 4
    def test_load_users(self):
        result = load_csv(source_file="/FileStore/tables/users_csv.gz", schema=users_schema)
        self.assertIsNotNone(result, "Users dataframe did not load successfully")
        self.assertIsInstance(result, DataFrame, "Result type is not of spark.sql.DataFrame")
        self.assertEqual(result.count(), 91616, "Number of records is not correct")

        coulmn_names = Counter(map(str.lower,
                                   ['Id', 'Reputation', 'CreationDate', 'DisplayName', 'LastAccessDate', 'AboutMe',
                                    'Views', 'UpVotes', 'DownVotes']))
        self.assertCountEqual(coulmn_names, Counter(map(str.lower, result.columns)),
                              "Missing column(s) or column name mismatch")
    # test 5
    def test_save_dfs(self):
        dfs = [("/FileStore/tables/users_csv.gz", users_schema, "users"),
               ("/FileStore/tables/badges_csv.gz", badges_schema, "badges"),
               ("/FileStore/tables/comments_csv.gz", comments_schema, "comments"),
               ("/FileStore/tables/posts_csv.gz", posts_schema, "posts")
               ]

        for i in dfs:
            df = load_csv(source_file=i[0], schema=i[1])
            save_df(df, i[2])

Success

.....
----------------------------------------------------------------------
Ran 5 tests in 31.903s

OK
Out[5]: <unittest.runner.TextTestResult run=5 errors=0 failures=0>

#### Subtask 4: answering to questions about Spark related concepts

Please write a short description for the terms below---one to two short paragraphs for each term. Don't copy-paste; instead, write your own understanding.

1. What do the terms 'Spark Application', 'SparkSession', 'Transformations', 'Action', and 'Lazy Evaluation' mean in the context of Spark?

Write your descriptions in the next cell.

When we write any job in spark, it is called a spark application. It refers to any program that use the open-source distributed data processing framework, Apache Spark. In a spark application, we have two important components, the driver and the executors. The driver makes sure that the work gets done correctly. It defines the computation logic, divides the work into tasks, and delegates it to the executors, who performs the tasks. The executors are working nodes in the Spark Cluster responsible for performing the tasks assigned by the driver program. First thing you need to do when writing your spark application is to create a sparkSession in the driver program. The sparkSession provides an interface with spark functionality and creates the connection to the Cluster Manager. When we run our code it goes to the cluster manager, which will grant resources the application needs to complete the work. 

Resilient Distributed Datasets (RDDs) supports two types of operations; Transformations and Actions. Transformations such as map() and filter() are used to modify the data and create new datasets. We call Transformations lazy operations because of the way they are not immediately executed when defined. Actions such as count() and collect() are used to extract a result from the dataset and return this in form of a value. These operations trigger the actual computation of the Spark application. 

When you call a Transformation on a RDD or a DataFrame, it will not be executed before an Action is called. This is a fundamental Spark concept called Lazy Evaluation, and holds several advantages when it comes to performance. When a transformation is called, Spark makes a Directed Acyclic Graph (DAG) which defines the order of transformations to be applied, a logical execution plan. This plan will only be evaluated and executed when we call an Action operation. By evaluating the plan, Spark finds the optimal plan before execution.